# Decomposition

In [ ]:
from utils import (
    load_dataset,
    get_images,
    transform_points,
    predict,
    SCALING_FACTOR
)
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
filename = 'data/intrinsics.json'

with open(filename, 'r') as file:
    data = json.load(file)
K = data['mtx']
dist = data['dist']
K = np.array(K)
dist = np.array(dist)

In [ ]:
def get_camera_position(pv_img, tv_img='IMG_00'):
    reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
    img_pv, img_tv = get_images(pv_img, tv_img)

    reference_pts_pv_arr, reference_pts_tv_arr, validation_pts_pv_arr, validation_pts_tv_arr = transform_points(
        reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv)
    reference_pts_pv_arr = reference_pts_pv_arr[:,:2].astype(np.float32)
    reference_pts_tv_arr = reference_pts_tv_arr[:,:2].astype(np.float32) 

    image_points = reference_pts_pv_arr
    object_points = reference_pts_tv_arr
    object_points = np.pad(
        object_points, 
        ((0, 0), (0, 1)), 
        mode = 'constant', 
        constant_values = 0
    )
    
    retval, rvec, tvec = cv2.solvePnP(
        object_points, 
        image_points, 
        K,
        dist,
        flags=cv2.SOLVEPNP_ITERATIVE
    )
    rotM = cv2.Rodrigues(rvec)[0]
    camera_position = np.matmul(-rotM.T, tvec) 
    return camera_position
pos_0026 = get_camera_position('IMG_01')
pos_0029 = get_camera_position('IMG_02')
pos_0032 = get_camera_position('IMG_03')
pos_0035 = get_camera_position('IMG_04')
pos_0038 = get_camera_position('IMG_05')
pos_0045 = get_camera_position('IMG_06')
pos_0049 = get_camera_position('IMG_07')
pos_0053 = get_camera_position('IMG_08')
pos_0061 = get_camera_position('IMG_09')
pos_0066 = get_camera_position('IMG_10')
pos_0067 = get_camera_position('IMG_11')
pos_0078 = get_camera_position('IMG_12')

In [ ]:
# <> Decomposition
pv_img = 'IMG_01'
tv_img = 'IMG_00'
reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
img_pv, img_tv = get_images(pv_img, tv_img)

reference_pts_pv_arr, reference_pts_tv_arr, validation_pts_pv_arr, validation_pts_tv_arr = transform_points(
    reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv)

h, _ = cv2.findHomography(
    reference_pts_pv_arr,
    reference_pts_tv_arr,
    # method = cv2.RANSAC,
    method = 0,
)
h_inv = np.linalg.inv(h)
num, Rs, Ts, Ns  = cv2.decomposeHomographyMat(h, K)
Ts

In [ ]:
preds = predict(validation_pts_pv_arr, h)
plt.imshow(img_tv)
plt.scatter(preds[:, 0], preds[:, 1])
plt.show()

In [ ]:
positions = np.concatenate(
    (pos_0026.T, 
     pos_0029.T, 
     pos_0032.T, 
     pos_0035.T, 
     pos_0038.T,
     pos_0045.T,
     pos_0049.T,
     pos_0053.T,
     pos_0061.T,
     pos_0066.T,
     pos_0067.T,
     pos_0078.T), 
    axis=0)
# I checked all positions manually, and they SEEM to be right! How to quantify it? 
# Height and angle we can do however.
# FoV Maybe also for visual proof.

In [ ]:
colors = np.arange(0, len(positions))
labels = [
    'IMG_01', 
    'IMG_02', 
    'IMG_03',
    'IMG_04',
    'IMG_05',
    'IMG_06',
    'IMG_07',
    'IMG_08',
    'IMG_09',
    'IMG_10',
    'IMG_11',
    'IMG_12'
]

_, img_tv = get_images('IMG_01', 'IMG_00')

from utils import load_data

validation_pts_tv = load_data(f'data/annotations/IMG_00_blue.txt')
reference_pts_tv = load_data(f'data/annotations/IMG_00_white.txt')

In [ ]:
import matplotlib.cm as cm
img_tv = cv2.imread('data/images/IMG_00.JPG', cv2.IMREAD_GRAYSCALE)
# img_pv = cv2.imread('data/images/DJI_0026.JPG', cv2.IMREAD_GRAYSCALE)
plt.imshow(img_tv, cmap='gray', alpha=0.5)
fc='red'
ec='black'

norm = plt.Normalize(min(colors), max(colors))
# Use a colormap (e.g., viridis, plasma, etc.)
colormap = plt.colormaps.get_cmap('viridis')

# Plot Reference and Validation Points
for name, pt in validation_pts_tv.items():
    plt.scatter(pt[0], pt[1], marker='X', color='steelblue', s=120, ec='black', alpha=0.5)
    if name in ['b8', 'b21', 'b24', 'b28']:
        x_offset, y_offset = 0, 0
        y_offset=200
        if name == 'b8':
            x_offset = 2000
            y_offset=-500
        if name == 'b28':
            y_offset=-200
            x_offset = 1000
        if name == 'b24':
            x_offset = 2200
        if name == 'b21':
            x_offset = 1000
            y_offset = 0
        plt.annotate(name, 
            xy=(pt[0], pt[1]),
            #xytext=(pt[0]-700, pt[1]+200),
            xytext=(1000+x_offset, pt[1]+y_offset),
            color='black',
            fontsize=14,
            arrowprops=dict(facecolor='steelblue', shrink=0.05),
            bbox=dict(facecolor='steelblue', edgecolor='black', boxstyle="round", alpha=0.7, lw=1.0, pad=0.2)
        )
        
for name, pt in reference_pts_tv.items():
    plt.scatter(pt[0], pt[1], marker='X', color='whitesmoke', s=120, ec='black', alpha=0.5)
    if name in ['w4', 'w5', 'w6', 'w7', 'w8', 'w9', 'w21', 'w22', 'w23', 'w26', 'w28']:
        x_offset, y_offset = 0, 150
        if name == 'w28':
            x_offset = -200
            y_offset = 150
        if name == 'w23':
            x_offset = -50
            y_offset = 150
        if name == 'w22':
            x_offset = -50
            y_offset = 120
        if name == 'w9':
            x_offset = -50
        plt.annotate(name, 
            xy=(pt[0]+x_offset, pt[1]+y_offset),
            # xytext=(pt[0]-700, pt[1]-200),
            color='black',
            fontsize=14,
            # arrowprops=dict(facecolor='white', shrink=0.05),
            bbox=dict(facecolor='white', edgecolor='black', boxstyle="round", alpha=0.7, lw=1.0, pad=0.2)
        )
    


# Plot each point individually with its corresponding label
for i in range(len(positions)):
    color = colormap(norm(colors[i]))
    plt.scatter(positions[i, 0], positions[i, 1], color=color, label=labels[i], marker='H', s=300,
               edgecolors='black')
    

# Create legend with unique labels
handles, unique_labels = plt.gca().get_legend_handles_labels()
unique_labels, unique_handles = zip(*dict(zip(unique_labels, handles)).items())
plt.legend(unique_handles, unique_labels)
plt.legend(loc='upper center', bbox_to_anchor=(0.23, 0.35),
          ncol=2, fancybox=False, shadow=False, framealpha=1, edgecolor='black')
plt.axis('off')

plt.tight_layout()
plt.savefig('output/camera_positions.png', dpi=300)
plt.show()

In [ ]:
heights = -positions[:, -1] / SCALING_FACTOR
for name, height in zip(labels, heights):
    print(f'{name}: {height:.2f}m')

### Rotations

In [ ]:
from scipy.spatial.transform import Rotation as R
def get_camera_rotation(pv_img, tv_img='IMG_00'):
    reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
    img_pv, img_tv = get_images(pv_img, tv_img)

    reference_pts_pv_arr, reference_pts_tv_arr, validation_pts_pv_arr, validation_pts_tv_arr = transform_points(
        reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv)
    reference_pts_pv_arr = reference_pts_pv_arr[:,:2].astype(np.float32)
    reference_pts_tv_arr = reference_pts_tv_arr[:,:2].astype(np.float32) 

    image_points = reference_pts_pv_arr
    object_points = reference_pts_tv_arr
    object_points = np.pad(
        object_points, 
        ((0, 0), (0, 1)), 
        mode = 'constant', 
        constant_values = 0
    )
    
    retval, rvec, tvec = cv2.solvePnP(
        object_points, 
        image_points, 
        K,
        dist
    )
    rotM = cv2.Rodrigues(rvec)[0]
    camera_position = np.matmul(-rotM.T, tvec) 

    rotation = R.from_matrix(rotM)
    
    # Convert the rotation to Euler angles (assuming 'zyx' order)
    euler_angles = rotation.as_euler('zyx', degrees=True)
    
    # Extract the individual angles
    yaw, pitch, roll = euler_angles
    return roll+90, yaw, pitch
rot_0026 = get_camera_rotation('IMG_01')
rot_0029 = get_camera_rotation('IMG_02')
rot_0032 = get_camera_rotation('IMG_03')
rot_0035 = get_camera_rotation('IMG_04')
rot_0038 = get_camera_rotation('IMG_05')
rot_0045 = get_camera_rotation('IMG_06')
rot_0049 = get_camera_rotation('IMG_07')
rot_0053 = get_camera_rotation('IMG_08')
rot_0061 = get_camera_rotation('IMG_09')
rot_0066 = get_camera_rotation('IMG_10')
rot_0067 = get_camera_rotation('IMG_11')
rot_0078 = get_camera_rotation('IMG_12')
rotations = [rot_0026[0], rot_0029[0], rot_0032[0], rot_0035[0], rot_0038[0], rot_0045[0], rot_0049[0], rot_0053[0], rot_0061[0], rot_0066[0], rot_0078[0]]

In [ ]:
for i, rotation in enumerate(rotations):
    print(f'IMG_{i}: {rotation.round(1)}')

# Deomposition on my own

In [ ]:
from scipy.spatial.transform import Rotation as SpicyR
def get_pose(pv_img):
    reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, 'IMG_00')
    reference_pts_pv_arr, reference_pts_tv_arr, validation_pts_pv_arr, validation_pts_tv_arr = transform_points(
        reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv)
    
    h, _ = cv2.findHomography(
        reference_pts_pv_arr,
        reference_pts_tv_arr,
        # method = cv2.RANSAC,
        method = 0,
    )
    h_inv = np.linalg.inv(h)

    H = h_inv.T
    h1 = H[0]
    h2 = H[1]
    h3 = H[2]
    K_inv = np.linalg.inv(K)
    L = 1 / np.linalg.norm(np.dot(K_inv, h1))
    h1, h2, h3 = h1*L, h2*L, h3*L
    #r1 = L * np.dot(K_inv, h1)
    #r2 = L * np.dot(K_inv, h2)
    r1 = np.dot(K_inv, h1)
    r2 = np.dot(K_inv, h2)
    r3 = np.cross(r1, r2)
    #T = L * (K_inv @ h3.reshape(3, 1))
    T = K_inv @ h3.reshape(3, 1)
    R = np.array([[r1], [r2], [r3]])
    R = np.reshape(R, (3, 3))
    T = -R@T
    rotation = SpicyR.from_matrix(R)
    euler_angles = rotation.as_euler('xyz', degrees=True)
    roll, pitch, yaw = euler_angles
    return roll-90, T
pitch, position = get_pose('IMG_01')
print(pitch)